# Establishing a database connection

In [59]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from urllib.parse import unquote

server_name   = "localhost"
database_name = "publictest"

connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};TrustServerCertificate=yes;UID=ITHS;PWD=ITHS;"
url_string        = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try:    
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=192.168.2.223:1433;DATABASE=publictest;TrustServerCertificate=yes;UID=ITHS;PWD=ITHS; 

Error while connecting to database:

(pyodbc.OperationalError) ('HYT00', '[HYT00] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)


# Setting up MetaData and declare tables

In [28]:
from sqlalchemy import MetaData, Table, Column, Integer, String, ForeignKey
metadata_obj = MetaData()

user_table = Table(
    "pythonUsers",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("username", String(30)),
    Column("password", String),
    Column("server_id", ForeignKey('pythonServers.id'), nullable=False)
)

server_table = Table(
    "pythonServers",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("servername", String),
    Column("IP", String(15))
)

for name, table in metadata_obj.tables.items():
    print(f"Table: {name}")

    for column in table.c:
        print(f"{column.name.ljust(15)}{column.type}")

    print()

Table: pythonUsers
id             INTEGER
username       VARCHAR(30)
password       VARCHAR
server_id      INTEGER

Table: pythonServers
id             INTEGER
servername     VARCHAR
IP             VARCHAR(15)



# Create all metadata tables

In [29]:
metadata_obj.create_all(engine)

print("Created tables:")

for table in metadata_obj.tables:
    print(table)

Created tables:
pythonUsers
pythonServers


# Drop all metadata tables

In [27]:
metadata_obj.drop_all(engine)

print("Dropped tables:")

for table in metadata_obj.tables:
    print(table)

Dropped tables:
pythonUsers
pythonServers


# Insert server data (multiple rows)

In [30]:
from sqlalchemy import insert
import random, string

servers = [
    {"servername": "Mario", "IP": "192.168.0.23"},
    {"servername": "Luigi", "IP": "192.168.0.185"},
    {"servername": "Yoshi", "IP": "192.168.0.15"},
    {"servername": "Bowser", "IP": "192.168.0.97"},
]

for server in servers:
    print(f"{server['servername'].ljust(15)}{server['IP']}")

statement = insert(server_table)
print(statement)

with engine.begin() as conn:
    conn.execute(statement, servers)

Mario          192.168.0.23
Luigi          192.168.0.185
Yoshi          192.168.0.15
Bowser         192.168.0.97
INSERT INTO "pythonServers" (id, servername, "IP") VALUES (:id, :servername, :IP)


# Insert user data (single row)

In [39]:
from sqlalchemy import insert
import random, string

username = input('Enter username:')
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(10))
server_id = random.randint(1, 4)

statement = insert(user_table).values(username=username, password=password, server_id=server_id)

print(statement)
print(statement.compile().params)

with engine.begin() as conn:
    result = conn.execute(statement)
    print(f"primary key value = {result.inserted_primary_key[0]}")

INSERT INTO "pythonUsers" (username, password, server_id) VALUES (:username, :password, :server_id)
{'username': 'Asajad', 'password': 'ahh6bzhs0l', 'server_id': 4}
primary key value = 9


# Select queries
In order to generate select queries we must first import select()

In [47]:
from sqlalchemy import select, or_, and_

Below are a few example of select queries:

### Select full table data

In [41]:
query = select(user_table)
print(query)

SELECT "pythonUsers".id, "pythonUsers".username, "pythonUsers".password, "pythonUsers".server_id 
FROM "pythonUsers"


### Projection

In [42]:
query = select(user_table.c.username, user_table.c.password, ("Hi " + user_table.c.username + "!").label('greeting'))
print(f"{query}\n\n{query.compile().params}")

SELECT "pythonUsers".username, "pythonUsers".password, :username_1 || "pythonUsers".username || :param_1 AS greeting 
FROM "pythonUsers"

{'username_1': 'Hi ', 'param_1': '!'}


### Selection

In [49]:
query = select(user_table).where(or_(user_table.c.username == 'Fredrik', user_table.c.server_id == 4))
print(f"{query}\n\n{query.compile().params}")

SELECT "pythonUsers".id, "pythonUsers".username, "pythonUsers".password, "pythonUsers".server_id 
FROM "pythonUsers" 
WHERE "pythonUsers".username = :username_1 OR "pythonUsers".server_id = :server_id_1

{'username_1': 'Fredrik', 'server_id_1': 4}


### Order by

In [51]:
query = select(user_table).order_by(user_table.c.server_id.asc(), user_table.c.username.desc()).limit(3)
print(query)

SELECT "pythonUsers".id, "pythonUsers".username, "pythonUsers".password, "pythonUsers".server_id 
FROM "pythonUsers" ORDER BY "pythonUsers".server_id ASC, "pythonUsers".username DESC
 LIMIT :param_1


### Joining tables

In [25]:
query = select(user_table.c.id, user_table.c.username, server_table.c.servername).join_from(user_table, server_table)
print(query)

SELECT "pythonUsers".id, "pythonUsers".username, "pythonServers".servername 
FROM "pythonUsers" JOIN "pythonServers" ON "pythonServers".id = "pythonUsers".server_id


### Grouping and aggregation

In [ ]:
from sqlalchemy import func

query = (
    select(server_table.c.servername, server_table.c.IP, func.count(user_table.c.id).label('Number of users'))
    .join_from(user_table, server_table)
    .group_by(server_table.c.servername, server_table.c.IP)
)

print(query)

# Fetch data

In [52]:
print(f"{query}\n\n{query.compile().params}\n")

with engine.connect() as conn:
    result = conn.execute(query)
    
    for column_name in result.keys():
        print(column_name.upper().ljust(20), end=' ')
    
    print()

    for row in result:
        for field in row:
            print(str(field).ljust(20), end=' ')
        
        print()

SELECT "pythonUsers".id, "pythonUsers".username, "pythonUsers".password, "pythonUsers".server_id 
FROM "pythonUsers" ORDER BY "pythonUsers".server_id ASC, "pythonUsers".username DESC
 LIMIT :param_1

{'param_1': 3}

ID                   USERNAME             PASSWORD             SERVER_ID            
3                    Anna                 8p6mnZqeSQ           1                    
7                    Petra                Jk5wOhWWeJ           2                    
5                    Karl                 fvhAWYNlN0           3                    


In [54]:
import pandas as pd

df = pd.read_sql_query(query, con=engine, index_col="id")

df

,username,password,server_id
id,,,
3,Anna,8p6mnZqeSQ,1
7,Petra,Jk5wOhWWeJ,2
5,Karl,fvhAWYNlN0,3


# Table reflection

In [55]:
reflected_table = Table("colors", metadata_obj, autoload_with=engine)

for column in reflected_table.c:
    print(f"{column.name.ljust(20)}{column.type}")


Name                NVARCHAR(20) COLLATE "Finnish_Swedish_CI_AS"
Code                NVARCHAR(7) COLLATE "Finnish_Swedish_CI_AS"
Red                 INTEGER
Green               INTEGER
Blue                INTEGER
hexcode             VARCHAR(7) COLLATE "Finnish_Swedish_CI_AS"


### Query reflected table

In [56]:
query = select(reflected_table).limit(10)
print(f"{query}\n\n{query.compile().params}\n")

SELECT colors."Name", colors."Code", colors."Red", colors."Green", colors."Blue", colors.hexcode 
FROM colors
 LIMIT :param_1

{'param_1': 10}

